In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.4 MB/s eta 0:00:00


##BERT Tokenizer

In [2]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
text = "BERT preprocessing is essential."
tokens = tokenizer.tokenize(text)

['bert', 'prep', '##ro', '##ces', '##sing', 'is', 'essential', '.']


In [4]:
tokenizer

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [3]:
print(tokens)

['bert', 'prep', '##ro', '##ces', '##sing', 'is', 'essential', '.']


In [ ]:
# BERT uses Word-piece Tokenization!
# BERT preprocessing is essential.

# ['bert', 'prep', '##ro', '##ces', '##sing', 'is', 'essential', '.']

##BERT model summary

In [5]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch

In [19]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
!pip install torchinfo

In [20]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

##BERT Model Input and Output Analysis

In [38]:
text = "This movie was amazing!"
inputs = tokenizer(text, return_tensors='pt')
outputs = model(**inputs)
predictions = torch.argmax(outputs.logits, dim=1)
print(predictions)

tensor([0])


In [39]:
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.0792, -0.3358]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [45]:
#understanding argmax:
a = torch.randint(4,(1,2))
print(a)
print("argmax of a:", torch.argmax(a, dim=1))

 #Returns the indices of the maximum value of all elements in the input tensor.

tensor([[2, 1]])
argmax of a: tensor([0])


In [22]:
inputs # input_ids, token_type_ids, attention_mask

#input_ids: each word has some id no.
#token_type_ids: one sequence ends another begins, these two sequence masked by different binary mask..Here only one context, so, only 0's present. (see next example)
#attention_mask: The attention mask is a binary tensor indicating the position of the padded indices so that the model does not attend to them. For the BertTokenizer, 1 indicates a value that should be attended to, while 0 indicates a padded value

{'input_ids': tensor([[ 101, 2023, 3185, 2001, 6429,  999,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [36]:
text = "This movie was amazing. What's the name of the movie?"
text2 = "I'm Ranjit"
inputs = tokenizer([text, text2], return_tensors='pt', padding=True)
inputs

{'input_ids': tensor([[  101,  2023,  3185,  2001,  6429,  1012,  2054,  1005,  1055,  1996,
          2171,  1997,  1996,  3185,  1029,   102],
        [  101,  1045,  1005,  1049,  2743, 18902,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}

##BERT's Attention Mechanism

In [46]:
#BERT's Attention:

text = "BERT's attention mechanism is fascinating."
inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
outputs = model(**inputs, output_attentions=True)

attention_weights = outputs.attentions
print(attention_weights)

(tensor([[[[7.0430e-02, 4.7734e-02, 4.5779e-02,  ..., 9.7265e-02,
           1.3927e-01, 3.5823e-01],
          [3.6679e-02, 8.6646e-02, 4.3685e-02,  ..., 2.3392e-01,
           3.7266e-02, 6.5047e-02],
          [8.5532e-02, 1.3269e-01, 3.7626e-02,  ..., 1.2145e-01,
           1.1870e-01, 1.3301e-01],
          ...,
          [5.4352e-02, 1.2452e-01, 1.4481e-01,  ..., 1.2920e-01,
           1.0262e-01, 1.0715e-01],
          [8.2242e-02, 1.0184e-01, 9.4826e-02,  ..., 9.6448e-02,
           1.5859e-01, 1.1109e-01],
          [9.6005e-02, 6.8182e-02, 6.3413e-02,  ..., 1.3608e-01,
           1.7070e-01, 1.8861e-01]],

         [[6.8129e-01, 9.2499e-03, 2.2069e-01,  ..., 5.5413e-03,
           3.5570e-02, 7.2794e-03],
          [2.3929e-01, 5.2402e-02, 8.9530e-02,  ..., 1.1060e-01,
           1.2288e-01, 1.3781e-01],
          [1.9622e-02, 1.6688e-01, 1.2318e-02,  ..., 2.9411e-01,
           1.1144e-01, 3.4867e-02],
          ...,
          [1.6697e-01, 9.8040e-02, 3.0382e-02,  ..., 7.042

In [58]:
print(len(attention_weights)) # 10 x 10 x 12 x 1

12


In [59]:
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.0128, -0.3319]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=(tensor([[[[7.0430e-02, 4.7734e-02, 4.5779e-02,  ..., 9.7265e-02,
           1.3927e-01, 3.5823e-01],
          [3.6679e-02, 8.6646e-02, 4.3685e-02,  ..., 2.3392e-01,
           3.7266e-02, 6.5047e-02],
          [8.5532e-02, 1.3269e-01, 3.7626e-02,  ..., 1.2145e-01,
           1.1870e-01, 1.3301e-01],
          ...,
          [5.4352e-02, 1.2452e-01, 1.4481e-01,  ..., 1.2920e-01,
           1.0262e-01, 1.0715e-01],
          [8.2242e-02, 1.0184e-01, 9.4826e-02,  ..., 9.6448e-02,
           1.5859e-01, 1.1109e-01],
          [9.6005e-02, 6.8182e-02, 6.3413e-02,  ..., 1.3608e-01,
           1.7070e-01, 1.8861e-01]],

         [[6.8129e-01, 9.2499e-03, 2.2069e-01,  ..., 5.5413e-03,
           3.5570e-02, 7.2794e-03],
          [2.3929e-01, 5.2402e-02, 8.9530e-02,  ..., 1.1060e-01,
           1.2288e-01, 1.3781e-01],
          [1.9622e-02, 1.6688e-01, 1.2318e-0

##Masked Language Model Nature

In [62]:
# Masked Language Model

from transformers import BertForMaskedLM, BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

text = "BERT is a powerful language model."
inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, add_special_tokens=True)
outputs = model(**inputs, labels=inputs['input_ids'])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [63]:
outputs

MaskedLMOutput(loss=tensor(4.0929, grad_fn=<NllLossBackward0>), logits=tensor([[[ -6.7831,  -6.7451,  -6.7206,  ...,  -5.9819,  -5.9509,  -4.2456],
         [ -6.7414,  -6.6768,  -6.7698,  ...,  -6.5734,  -6.2763,  -4.5366],
         [ -9.3242,  -8.8146,  -9.2216,  ...,  -8.0251,  -6.2650,  -6.6039],
         ...,
         [-11.1757, -11.2235, -11.4956,  ..., -10.4808,  -9.2249,  -8.7422],
         [ -9.8364,  -9.7097,  -9.9151,  ...,  -7.5149,  -8.9583,  -6.4237],
         [ -9.6640, -10.1441,  -9.9603,  ...,  -7.4035,  -8.7196,  -6.1262]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)

In [66]:
loss = outputs.loss
print(loss)

tensor(4.0929, grad_fn=<NllLossBackward0>)


##BERT Embeddings

In [67]:
# Contextual Word Embeddings: based on context; creates id for word (smae word issues resolved)
# WordPiece Tokenization: handles complex vocabs (complex word made up of subwords)
# Positional Encodings: added to embeddings, give position of each word


from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

text = "BERT embeddings are fascinating."
inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, add_special_tokens=True)
outputs = model(**inputs)

word_embeddings = outputs.last_hidden_state
print(word_embeddings)

tensor([[[-0.1094,  0.2831, -0.2219,  ..., -0.4116,  0.2604,  0.4303],
         [ 0.3717,  0.0461,  0.3014,  ..., -0.2603, -0.2263,  0.0782],
         [-0.2072,  0.3525, -0.2668,  ..., -0.0065, -0.6920, -0.0067],
         ...,
         [ 0.5834,  0.4004,  0.4881,  ..., -0.1181,  0.2630, -0.1225],
         [ 0.0716, -0.2377, -0.3874,  ...,  0.2981,  0.0436, -0.6352],
         [ 0.8747,  0.4112, -0.3423,  ...,  0.2614, -0.4867, -0.2898]]],
       grad_fn=<NativeLayerNormBackward0>)


In [68]:
inputs

{'input_ids': tensor([[  101, 14324,  7861,  8270,  4667,  2015,  2024, 17160,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [69]:
outputs

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.1094,  0.2831, -0.2219,  ..., -0.4116,  0.2604,  0.4303],
         [ 0.3717,  0.0461,  0.3014,  ..., -0.2603, -0.2263,  0.0782],
         [-0.2072,  0.3525, -0.2668,  ..., -0.0065, -0.6920, -0.0067],
         ...,
         [ 0.5834,  0.4004,  0.4881,  ..., -0.1181,  0.2630, -0.1225],
         [ 0.0716, -0.2377, -0.3874,  ...,  0.2981,  0.0436, -0.6352],
         [ 0.8747,  0.4112, -0.3423,  ...,  0.2614, -0.4867, -0.2898]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-0.8795, -0.4697, -0.9591,  0.8022,  0.6837, -0.1953,  0.8998,  0.3850,
         -0.9031, -1.0000, -0.5993,  0.9340,  0.9694,  0.7847,  0.9166, -0.7953,
         -0.3018, -0.6575,  0.4096, -0.4689,  0.7108,  1.0000, -0.2501,  0.5017,
          0.6107,  0.9906, -0.7811,  0.9156,  0.9502,  0.7427, -0.7152,  0.3800,
         -0.9855, -0.3006, -0.9457, -0.9903,  0.6001, -0.6676, -0.1011, -0.0583,
         -0.8776,  0.4252,  1.00

##Access intermediate layers

In [70]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

text = "Advanced fine-tuning with BERT."
inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
outputs = model(**inputs, output_hidden_states=True)

intermediate_layer = outputs.hidden_states[6]  # 7th layer (total 12 layers)
print(intermediate_layer)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([[[-0.3527, -1.1815, -0.3027,  ..., -0.1794,  0.0872,  0.5573],
         [-0.3768,  0.1754,  0.3395,  ..., -0.0359,  0.2162, -1.2251],
         [ 1.8406, -0.6458,  0.5841,  ..., -0.7345,  0.7542, -0.1614],
         ...,
         [ 1.0414, -0.7009,  1.0362,  ...,  1.0581, -0.3068, -1.4171],
         [-0.8934, -0.8139, -0.3154,  ..., -0.3933, -0.6383,  0.0522],
         [ 0.0143, -0.0423, -0.0131,  ...,  0.0044, -0.0140, -0.0394]]],
       grad_fn=<NativeLayerNormBackward0>)


##Contextual word Embeddings of BERT and RoBERTa

In [81]:
from transformers import RobertaTokenizer, RobertaModel
import torch

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')

text = "RoBERTa is an advanced variant of BERT."
inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
outputs = model(**inputs)

embeddings = outputs.last_hidden_state
print(embeddings)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([[[-0.0640,  0.1073, -0.0181,  ..., -0.0383, -0.0555, -0.0151],
         [-0.0647,  0.0450, -0.0528,  ...,  0.0814, -0.1633, -0.0284],
         [ 0.0398,  0.0657, -0.1046,  ..., -0.1364, -0.0650,  0.0560],
         ...,
         [ 0.0273,  0.0271, -0.0042,  ..., -0.1554, -0.0037,  0.1105],
         [-0.0560,  0.1078, -0.0385,  ..., -0.0619, -0.0614, -0.0363],
         [ 0.0064,  0.1383,  0.0011,  ...,  0.0994, -0.0593,  0.0169]]],
       grad_fn=<NativeLayerNormBackward0>)


In [82]:
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

text = "RoBERTa is an advanced variant of BERT."
inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, add_special_tokens=True)
outputs = model(**inputs)

word_embeddings = outputs.last_hidden_state
print(word_embeddings)

tensor([[[-0.5245, -0.2653,  0.0204,  ..., -0.1743, -0.1021,  0.6587],
         [ 0.3463, -0.3660,  0.2527,  ...,  0.0454,  0.3741,  0.3258],
         [-0.8831, -0.6733,  0.3973,  ..., -0.1721,  0.0040,  0.8362],
         ...,
         [ 0.5319, -0.2398,  0.3179,  ...,  0.2273, -0.0825, -0.0774],
         [ 0.3819,  0.2676, -0.5101,  ...,  0.2828, -0.5341, -0.4512],
         [-0.4393,  0.2006,  0.2160,  ...,  0.6901, -0.6010, -0.2884]]],
       grad_fn=<NativeLayerNormBackward0>)
